In [2]:
import pandas as pd

In [3]:
df = pd.read_excel("K8s Compliance Scanner Evaluation.xlsx", sheet_name="Detailed")

In [4]:
df

,ID,Mechanism,K8s Layer,Resource Kind,Control Type,Active Configuration,Name,Control,Path To Check,Action,...,Checkov,datree,Kubestriker,Kube-Score,KubeLinter,Terrascan,KICS,Starboard (defsec),Snyk,Color code explanation
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,supports custom rules,NaN,NaN,...,yes,yes,no,no,yes,yes (OPA),yes (OPA),yes (OPA),yes,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autofix,NaN,NaN,...,no,no,no,no,no,no,no,no,yes,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,max. 1000 files/month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Requires sign-up to manage policies via their ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,white: no control found
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test scanners on difference `runAsUser` > 1k a...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kube-score: test if it validates only PodTempl...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test settings on Pod.SecurityContext vs Contai...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
MECHANISM = "Mechanism"
K8S_LAYER = "K8s Layer"
CONTROL = "Control"

In [6]:
info_cols = [MECHANISM, K8S_LAYER, CONTROL] 

In [7]:
fw_cols = ["CIS (v1.8)", "NSA-CISA", "K8s STIG", "BSI",	"K8s Security Checklist", "PCI DSS", "Mitre", "OWASP"]

In [8]:
first_tool_idx = list(df.columns).index("Kube-bench")
tool_cols = list(df.columns)[first_tool_idx:-1]  # last column is color code explanation

In [9]:
def filter_relevant_controls(df: pd.DataFrame) -> pd.DataFrame:
    # valid rows are all with have a named control and either Mechanism or layer are defined (ideally both of course)
    return df[df[CONTROL].notnull() & (df[MECHANISM].notnull() | df[K8S_LAYER].notnull())]

In [10]:
def agg(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    return df[[MECHANISM, K8S_LAYER]+cols].groupby([MECHANISM, K8S_LAYER]).count().sum(axis=1).unstack(fill_value=0)

In [32]:
def process_df(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    # merge all selected columns with the necessary columns for robustness (or querying just unique controls)
    return (df[list(set(info_cols+cols))]
            .pipe(filter_relevant_controls)
            .pipe(agg, cols=cols)
            )

In [31]:
df_unique_ctrls = process_df(df, cols=[CONTROL])
df_unique_ctrls

K8s Layer,Application,Cluster Network,Control Plane,Data,Multi-Tenancy,Node,Workload
Mechanism,,,,,,,
Admission Control,0,5,13,1,0,1,33
Authorization,0,0,29,0,0,5,28
Data Security,0,2,9,10,0,3,1
Hardening,0,0,1,0,0,11,0
Logging and Auditing,0,0,22,0,0,1,5
Reliability,0,0,0,0,0,0,26
Runtime,2,0,0,0,0,1,1
Segregation,0,8,0,0,4,0,6
SupplyChain,2,0,0,0,0,0,0


In [25]:
df_fws = process_df(df, cols=fw_cols)
df_tools = process_df(df, cols=tool_cols)

In [26]:
df_all = df_fws.add(df_tools, fill_value=0)
df_all

K8s Layer,Application,Cluster Network,Control Plane,Data,Multi-Tenancy,Node,Workload
Mechanism,,,,,,,
Admission Control,0,15,26,1,0,2,146
Authorization,0,0,67,0,0,16,146
Data Security,0,4,23,39,0,7,1
Hardening,0,0,1,0,0,12,0
Logging and Auditing,0,0,38,0,0,3,8
Reliability,0,0,0,0,0,0,123
Runtime,3,0,0,0,0,5,5
Segregation,0,40,0,0,8,0,24
SupplyChain,19,0,0,0,0,0,0


In [27]:
print(df_all.to_latex(caption="An overview of all distilled checks across all frameworks and tools", bold_rows=True, ))

\begin{table}
\caption{An overview of all distilled checks across all frameworks and tools}
\begin{tabular}{lrrrrrrr}
\toprule
K8s Layer & Application & Cluster Network & Control Plane & Data & Multi-Tenancy & Node & Workload \\
Mechanism &  &  &  &  &  &  &  \\
\midrule
\textbf{Admission Control} & 0 & 15 & 26 & 1 & 0 & 2 & 146 \\
\textbf{Authorization} & 0 & 0 & 67 & 0 & 0 & 16 & 146 \\
\textbf{Data Security} & 0 & 4 & 23 & 39 & 0 & 7 & 1 \\
\textbf{Hardening} & 0 & 0 & 1 & 0 & 0 & 12 & 0 \\
\textbf{Logging and Auditing} & 0 & 0 & 38 & 0 & 0 & 3 & 8 \\
\textbf{Reliability} & 0 & 0 & 0 & 0 & 0 & 0 & 123 \\
\textbf{Runtime} & 3 & 0 & 0 & 0 & 0 & 5 & 5 \\
\textbf{Segregation} & 0 & 40 & 0 & 0 & 8 & 0 & 24 \\
\textbf{SupplyChain} & 19 & 0 & 0 & 0 & 0 & 0 & 0 \\
\textbf{Vulnerability Management} & 0 & 2 & 3 & 5 & 0 & 2 & 11 \\
\textbf{Workload Security} & 0 & 11 & 0 & 35 & 0 & 1 & 288 \\
\bottomrule
\end{tabular}
\end{table}



In [33]:
def to_latex(df: pd.DataFrame) -> str:
    tex = df.style.set_caption("An overview of all distilled checks from all grey literature and tools").to_latex()
    tex = tex.replace("table", "table*")  # make the table span 2 columns; there is no option yet for the Styler to do this directly
    return tex

In [35]:
print(to_latex(df_unique_ctrls))

\begin{table*}
\caption{An overview of all distilled checks from all grey literature and tools}
\begin{tabular}{lrrrrrrr}
K8s Layer & Application & Cluster Network & Control Plane & Data & Multi-Tenancy & Node & Workload \\
Mechanism &  &  &  &  &  &  &  \\
Admission Control & 0 & 5 & 13 & 1 & 0 & 1 & 33 \\
Authorization & 0 & 0 & 29 & 0 & 0 & 5 & 28 \\
Data Security & 0 & 2 & 9 & 10 & 0 & 3 & 1 \\
Hardening & 0 & 0 & 1 & 0 & 0 & 11 & 0 \\
Logging and Auditing & 0 & 0 & 22 & 0 & 0 & 1 & 5 \\
Reliability & 0 & 0 & 0 & 0 & 0 & 0 & 26 \\
Runtime & 2 & 0 & 0 & 0 & 0 & 1 & 1 \\
Segregation & 0 & 8 & 0 & 0 & 4 & 0 & 6 \\
SupplyChain & 2 & 0 & 0 & 0 & 0 & 0 & 0 \\
Vulnerability Management & 0 & 2 & 3 & 2 & 0 & 2 & 3 \\
Workload Security & 0 & 2 & 0 & 5 & 0 & 1 & 34 \\
\end{tabular}
\end{table*}

